In [ ]:
import os
import random
import numpy as np
import ctypes
c_longlong = ctypes.c_longlong

def size_without_header(file_path):
    # print("opening", file_path)
        # Read the first lines of the file until we reach some non-text (binary) data
    try:
        with open(file_path, "rb") as file:
            # print("opened", file_path)
            # Read line by line until an exception is thrown or the end of file is reached
            header = ""
            
            while True:
                line = file.readline()
                # print(line)
                if line == "":
                    break
                header += line.decode("utf-8")
            
    
    except Exception as e:
        pass
        
        # Now we know the header, we can get the size of the file
    with open(file_path, "rb") as file:
        file.seek(0, os.SEEK_END)
        file_size = file.tell()
        # Subtract the size of the header from the file size
        file_size -= len(header)
    return file_size
        
        
        

gt_directory_path = '/media/andrew/One Touch/HMNET2/experiments/detection/data/gen1/source/detection_dataset_duration_60s_ratio_1.0/test'
adder_shrunk_directory_root = '/media/andrew/One Touch/HMNET_ADDER_data'
output_directory_root = '/media/andrew/One Touch/HMNET_ADDER_data'

gt_file_list = os.listdir(gt_directory_path)
# Filter to just those ending int .dat
gt_file_list = [filename for filename in gt_file_list if filename.endswith('.dat')]

for i in [3,6,9]:
    print("CRF: ", i)
    adder_directory_path = os.path.join(output_directory_root, f'gen{i}_dat')
    output_directory_path = os.path.join(output_directory_root, f'gen{i}_random_shrink')
    os.makedirs(output_directory_path, exist_ok=True)
    already_done_files = os.listdir(output_directory_path)
    
    for filename in gt_file_list:
        if filename in already_done_files:
            continue
        else:
            print(filename)
            # Do the thing
            # 1. Load the file
            # Get the size of the ADDER version in bytes
            adder_file_path = os.path.join(adder_directory_path, filename)
            # If the ADDER file doesn't exist, then we can't shrink it
            if not os.path.exists(adder_file_path):
                print(f'ADDER file {filename} does not exist. Skipping')
                continue
            
            adder_file_size = size_without_header(adder_file_path)
            num_adder_version_events = adder_file_size // 8
            # Get the size of the GT version in bytes
            gt_file_path = os.path.join(gt_directory_path, filename)
            gt_file_size = size_without_header(gt_file_path)
            num_gt_version_events = gt_file_size // 8
            
            # If the GT version is smaller than the ADDER version, then we can't shrink it
            if gt_file_size < adder_file_size:
                print(f'GT file {filename} is smaller than ADDER file {filename}. Skipping')
                continue
            # If the GT version is the same size as the ADDER version, then we don't need to shrink it
            elif gt_file_size == adder_file_size:
                print(f'GT file {filename} is the same size as ADDER file {filename}. Skipping')
                continue
            # If the GT version is larger than the ADDER version, then we can shrink it
            else:
                print(f'GT file {filename} is larger than ADDER file {filename}. Shrinking')
                # Randomly remove events from the GT version until it is the same size as the ADDER version
                # Load the GT file
                try:
                    with open(gt_file_path, "rb") as file:
                      # Read line by line until an exception is thrown or the end of file is reached
                      header = ""
                      for line in file:
                        header += line.decode("utf-8")
                
                except UnicodeDecodeError:
                    pass
                
                with open(gt_file_path, "rb") as file:
                    # seek to the end of the header
                    file.seek(len(header))
                    # Read the rest of the file
                    data = file.read()
                # Convert the data to a list of 8-byte integers
                data = [data[i:i+8] for i in range(0, len(data), 8)]
                print(type(data[0]))

                
                print("Need to remove", len(data) - num_adder_version_events, "events")
                
                # Randomly remove events until the file is the same size as the ADDER version
                
                data_array = np.array(data, dtype=object)

                # Select N random indices using NumPy's random.choice
                random_indices = np.random.choice(len(data_array), size=len(data_array)-num_adder_version_events, replace=False)
            
                # Sort the indices in ascending order to preserve removal order
                random_indices.sort()
            
                # Create a mask to select elements to keep (excluding the random indices)
                mask = np.ones(len(data_array), dtype=bool)
                mask[random_indices] = False
            
                # Use boolean indexing to filter the data and get the removed items
                modified_data = data_array[mask]
                removed_items = data_array[random_indices]
            
                # Convert the NumPy arrays back to lists if desired
                data = modified_data.tolist()

                
                
                # Convert the data back to bytes
                data = b''.join([x for x in data])
                # Save the file
                output_file_path = os.path.join(output_directory_path, filename)
                with open(output_file_path, 'wb') as f:
                    f.write(header.encode('utf-8'))
                    f.write(data)
                # Add the file to the already_done_files list
                
                
            
            # 2. Shrink the file
            # 3. Save the file
            # 4. Add the file to the already_done_files list
            # 5. Repeat
